In [ ]:
from pathlib import Path
import json

from jinja2 import Environment, FileSystemLoader, select_autoescape
import numpy as np
import requests
import yfinance as yf

import stock

In [ ]:
def format_number(value):
    return float("{:.2f}".format(value))

def render_bundle_js(env: Environment, ticker: str, output_dir: Path):
    template = env.get_template("bundle.js")
    history = yf.Ticker(ticker).history(period="1y", interval="1d")
    dates = [idx.strftime("%Y/%m/%d") for idx in history.index]
    daily_data = []
    volume_data = []
    for idx, row in enumerate(history.to_numpy()):
        daily_data.append([
            dates[idx], 
            format_number(row[0]), 
            format_number(row[3]), 
            format_number(row[1]),
            format_number(row[2]),
        ])
        volume_data.append(row[4])

    text = template.render(
        chart_id=ticker,
        daily_data=daily_data,
        volume_data=volume_data
    )

    output_dir.mkdir(exist_ok=True)
    output_path = output_dir / f"bundle_{ticker}.js"
    with open(output_path, "w") as f:
        f.write(text)
    return output_path

In [ ]:
watch_list_path = stock.DATA_DIR / "watch_list.csv"
with open(watch_list_path, "r") as f:
    watch_list = [txt.strip() for txt in f.readlines()[1:]]

In [ ]:
output_dir = Path("./tmp")
output_dir.mkdir(exist_ok=True)

env = Environment(
    loader=FileSystemLoader("../templates"),
    autoescape=select_autoescape()
)

# render index.html
template = env.get_template("index.html")
index_html_text = template.render(chart_ids=watch_list)
index_html_path = output_dir / "index.html"
with open(index_html_path, "w")  as f:
    f.write(index_html_text)

# render bundle javascript files
for ticker in watch_list:
    print(render_bundle_js(env, ticker, output_dir / "js"))
